In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 1.5 MB/s eta 0:00:00


In [18]:
#@title Authenticate and grant Colab access to your spreadsheets for this session

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [19]:
#@title Configure Gemini API key

#Access your Gemini API key

!pip install -q -U google-generativeai
import google.generativeai as genai
from google.colab import userdata

gemini_api_secret_name = 'GOOGLE_API_KEY'

try:
  GOOGLE_API_KEY=userdata.get(gemini_api_secret_name)
  genai.configure(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
   print(f'''Secret not found

This expects you to create a secret named {gemini_api_secret_name} in Colab

Visit https://makersuite.google.com/app/apikey to create an API key

Store that in the secrets section on the left side of the notebook (key icon)

Name the secret {gemini_api_secret_name}''')
   raise e
except userdata.NotebookAccessError as e:
  print(f'''You need to grant this notebook access to the {gemini_api_secret_name} secret in order for the notebook to access Gemini on your behalf.''')
  raise e
except Exception as e:
  # unknown error
  print(f"There was an unknown error. Ensure you have a secret {gemini_api_secret_name} stored in Colab and it's a valid key from https://makersuite.google.com/app/apikey")
  raise e

model = genai.GenerativeModel('gemini-pro')

In [21]:
#@title Enter the name of a spreadsheet to import

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1z0DduKcOfwMOEiUWxvMCFAUEZFubQK3SretL0q0l5O0/edit?usp=sharing"

worksheet = gc.open_by_url(spreadsheet_url).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows[0:5])

[['Name', 'Relationship', 'Style'], ['Rohan', 'Brother', 'Sincere'], ['Venky', 'Brother', 'Funny'], ['Husnain', 'Colleague', 'Wish'], ['Sufyan', 'Friend', 'joy']]


In [22]:
#@title Use Gemini to suggest what to say on a Christmas note

rows[0].append('Note')

#skip the title row
for row in rows[1:]:
  prompt = "I'm writing Christmas cards. Can you suggest a short meaningful paragraph of text for a Christmas note? I'm writing a note to %s my %s and I'd like the style to be %s." % (row[0], row[1], row[2])

  response = model.generate_content(prompt)
  row.append(response.text)

#print out the results nicely
from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

to_markdown(''.join([row[3]+'\n\n' for row in rows[1:]]))

> Dear Rohan, my dearest brother, as this festive season envelops us with warmth and cheer, I extend my heartfelt Christmas greetings to you. May this Christmas bring you an abundance of joy, peace, and love. As we gather around the Christmas tree, let us cherish the cherished memories we share and forge new ones that will last a lifetime. Embrace the magic of this season and let it reignite the bond we hold so dear. Merry Christmas, my brother!
> 
> "Dear Venky,
> 
> So here we are again at Christmas. I can hardly believe another year has flown by. It's like the older you get, the faster time goes, huh? It amazes me how we've managed to make it through another twelve months, especially given all the crazy stuff that's been going on in the world. It's a reminder that we are capable of weathering any storm, as long as we have each other. You are my constant and most valued treasure. I hope you're aware of how much you mean to me and that you're worth more to me than a room full of presents. Jokes aside, I sincerely wish this Christmas brings you as much joy and contentment as you bring into my life. You're the best brother anyone could ask for, except for the occasional nose-picking. But hey, nobody's perfect, right? Merry Christmas, Venky! Love you to the North Pole and back!"
> 
> Dear Husnain,
> Wishing you a Merry Christmas and a fantastic holiday season! I am so grateful to have a wonderful colleague like you, who makes work so much more enjoyable. Your positive attitude and strong work ethic are an inspiration to us all. I hope you have an amazing time with your loved ones, full of warmth, laughter, and joy. May the magic of Christmas bring you happiness, success, and fulfillment in the coming year. Warmest wishes to you and your family.
> 
> Best regards,
> 
> As the cheerful season embraces us, let's celebrate the love and goodwill that Christmas brings. Sufyan, you are a true friend, and every moment with you brings joy to my heart. May this Christmas be a joyful ride for us, creating memories that will last forever. Let's be thankful for the blessings we have and may our friendship continue to grow and spread warmth, just like the spirit of Christmas. Best wishes for a memorable Christmas and a bright, blessed New Year!
> 


In [23]:
#@title Write the notes back to your spreadsheet

#this chooses which column to write to
col = len(rows[0])
start_cell = chr(col + 64) + '1'

#write the column
try:
  worksheet.update(start_cell, [[row[3]] for row in rows])
except Exception as e:
  print('There was an error writing to your spreadsheet. Make sure you have permissions to edit the spreadsheet.')
  raise e